1. *Самостоятельно повторить tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
import pandas as pd

Наши новости

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR" -O articles.csv && rm -rf /tmp/cookies.txt 
!wget 'https://drive.google.com/uc?export=download&id=1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a' -O users_articles.csv

--2022-03-23 09:39:20--  https://docs.google.com/uc?export=download&confirm=t&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR
Resolving docs.google.com (docs.google.com)... 172.217.203.102, 172.217.203.100, 172.217.203.101, ...
Connecting to docs.google.com (docs.google.com)|172.217.203.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qdgapk904gafk0k0q4mohs0drf8qgprb/1648028325000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download [following]
--2022-03-23 09:39:20--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qdgapk904gafk0k0q4mohs0drf8qgprb/1648028325000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 172.217.203.132, 2607:f8b0:400c:c07::84
Connecting to doc-0s-c0-docs.googleusercontent.com (doc-

In [ ]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [ ]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [ ]:
!pip install razdel pymorphy2 pyLDAvis

In [ ]:
# предобработка текстов
import re
import numpy as np
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize  # сегментация русскоязычного текста на токены и предложения https://github.com/natasha/razdel
import pymorphy2  # Морфологический анализатор

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

151


In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c' -O stopwords.txt

--2022-03-23 09:39:34--  https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c
Resolving drive.google.com (drive.google.com)... 173.194.217.138, 173.194.217.101, 173.194.217.102, ...
Connecting to drive.google.com (drive.google.com)|173.194.217.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5vdm5cbleupm6hrplt1qhftshfr1ddg8/1648028325000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download [following]
--2022-03-23 09:39:35--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5vdm5cbleupm6hrplt1qhftshfr1ddg8/1648028325000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 172.217.203.132, 2607:f8b0:400c:c07::84
Connecting to doc-0s-c0-docs.googleusercontent.com (doc-0s-c0

In [ ]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
    
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [ ]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):    
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [ ]:
from tqdm import tqdm
tqdm.pandas()

# Запускаем очистку текста. Будет долго...
news['title'] = news['title'].progress_apply(lambda x: clean_text(x))

  0%|          | 0/27000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Possible nested set at position 39
  
100%|██████████| 27000/27000 [00:41<00:00, 647.77it/s] 


In [ ]:
news['title'].iloc[:10]

0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
2    форвард авангарда томаш заборский прокомментир...
3    главный тренер кубани юрий красножан прокоммен...
4    решением попечительского совета владивостокско...
5    ио главного тренера вячеслав буцаев прокоммент...
6    запорожский металлург дома потерпел разгромное...
7    сборная сша одержала победу над австрией со сч...
8    бывший защитник сборной россии дарюс каспарайт...
9    полузащитник цска зоран тошич после победы над...
Name: title, dtype: object

In [ ]:
# Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].progress_apply(lambda x: lemmatization(x))

100%|██████████| 27000/27000 [04:25<00:00, 101.66it/s]


А теперь в 3 строчки обучим нашу модель

In [ ]:
# сформируем список наших текстов
texts = list(news['title'].values)

# Создадим корпус из списка с текстами
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Запускаем обучение

In [ ]:
N_topic = 20  # можно было менять

In [ ]:
from gensim.models import LdaModel

# Обучаем модель на корпусе
lda = LdaModel(common_corpus, num_topics=N_topic, id2word=common_dictionary, passes=2)  # можно было менять

In [ ]:
from gensim.test.utils import datapath

# Сохраняем модель на диск
temp_file = datapath("model.lda")
lda.save(temp_file)

In [ ]:
# Загружаем обученную модель с диска
lda = LdaModel.load(temp_file)

In [ ]:
# Создаем новый корпус документов, которые раньше не видели
other_texts = list(news['title'].iloc[:3])
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(2, 0.054446954),
 (7, 0.044650584),
 (12, 0.510388),
 (15, 0.08303707),
 (18, 0.040467285),
 (19, 0.24951014)]

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [ ]:
x = lda.show_topics(num_topics=N_topic, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# Печатаем только слова
for topic, words in topics_words:
    print(f"topic_{topic}: " + " ".join(words))

topic_0: ракета полёт учёный орбита пассажир двигатель км
topic_1: россия российский армия погибнуть запуск территория китай
topic_2: эксперт всё земля тело смерть век рак
topic_3: млрд рубль млн руб цена фонд рынок
topic_4: сотрудник россия уголовный орган убийство лицо задержать
topic_5: луна женщина эксперимент мужчина asa ген тыс
topic_6: заболевание врач организм обнаружить клетка британский около
topic_7: всё ребёнок наука учёный уровень университет турист
topic_8: военный сша россия нефть американский война российский
topic_9: крым пенсия ветер крымский агентство полуостров корея
topic_10: россия российский банк правительство вопрос экономика глава
topic_11: доход миссия писать спрос налог производитель великобритания
topic_12: день земля температура поверхность всё солнце планета
topic_13: исследование учёный сша исследователь журнал операция опубликовать
topic_14: самолёт корабль экипаж станция море восток boei
topic_15: космос медведев фильм гражданин космический рейтинг план

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [ ]:
def get_lda_vector(lda, text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]

    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [ ]:
get_lda_vector(lda, news['title'].iloc[0])

array([0.        , 0.02833068, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.8570593 , 0.        , 0.10715393, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [ ]:
topic_matrix = pd.DataFrame([get_lda_vector(lda, text) for text in news['title'].values])
topic_matrix.columns = [f'topic_{i}' for i in range(N_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+[f'topic_{i}' for i in range(N_topic)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,6,0.0,0.028323,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.857065,0.0,0.107156,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,4896,0.0,0.000000,0.00000,0.0,0.079059,0.0,0.0,0.000000,0.049736,...,0.000000,0.0,0.403872,0.0,0.0,0.000000,0.447821,0.0,0.000000,0.000000
2,4897,0.0,0.000000,0.05443,0.0,0.000000,0.0,0.0,0.044645,0.000000,...,0.000000,0.0,0.510397,0.0,0.0,0.083053,0.000000,0.0,0.040471,0.249505
3,4898,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.426420,0.000000,...,0.000000,0.0,0.374244,0.0,0.0,0.108661,0.000000,0.0,0.000000,0.080919
4,4899,0.0,0.369946,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.064303,0.0,0.000000,0.0,0.0,0.542778,0.000000,0.0,0.000000,0.000000


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [ ]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [ ]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(N_topic)]].values))

In [ ]:
doc_dict[293672]

array([0.        , 0.        , 0.        , 0.14430939, 0.08491388,
       0.        , 0.        , 0.16909412, 0.        , 0.        ,
       0.04236638, 0.04401259, 0.        , 0.        , 0.        ,
       0.        , 0.19239576, 0.30990788, 0.        , 0.        ])

In [ ]:
def get_user_embedding(user_articles_list, doc_dict, func):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, axis=0)
    return user_vector

In [ ]:
user_articles_list = users['articles'].iloc[33]

get_user_embedding(user_articles_list, doc_dict, np.mean)

array([0.00587311, 0.10440729, 0.03019478, 0.02853086, 0.22704806,
       0.02557321, 0.00238903, 0.07982895, 0.12432842, 0.        ,
       0.18064478, 0.02481279, 0.00198474, 0.06334368, 0.0371025 ,
       0.00898099, 0.01742829, 0.01996241, 0.00896084, 0.        ])

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [ ]:
FUNC = np.mean

In [ ]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.011767,0.015938,0.035789,0.056539,0.134000,0.035240,0.023151,0.078058,0.060247,...,0.030821,0.030748,0.004307,0.024716,0.023720,0.049912,0.098739,0.157074,0.092067,0.010097
1,u108690,0.000000,0.017720,0.141401,0.028406,0.164466,0.002050,0.027290,0.079994,0.078703,...,0.173746,0.033342,0.008906,0.057805,0.006797,0.029466,0.013553,0.031181,0.077544,0.005561
2,u108339,0.006854,0.136171,0.072904,0.044623,0.251824,0.004261,0.023074,0.063888,0.079558,...,0.043805,0.015415,0.025124,0.051542,0.026707,0.007515,0.029399,0.041229,0.056211,0.000000


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK' -O users_churn.csv

--2022-03-23 09:47:03--  https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK
Resolving drive.google.com (drive.google.com)... 173.194.217.113, 173.194.217.102, 173.194.217.101, ...
Connecting to drive.google.com (drive.google.com)|173.194.217.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3ko7cp1oe0uootrn1ku3qr2q2th4bpqk/1648028775000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download [following]
--2022-03-23 09:47:03--  https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3ko7cp1oe0uootrn1ku3qr2q2th4bpqk/1648028775000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download
Resolving doc-0k-c0-docs.googleusercontent.com (doc-0k-c0-docs.googleusercontent.com)... 172.217.203.132, 2607:f8b0:400c:c07::84
Connecting to doc-0k-c0-docs.googleusercontent.com (doc-0k-c0

In [ ]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.011767,0.015938,0.035789,0.056539,0.134000,0.035240,0.023151,0.078058,0.060247,...,0.030748,0.004307,0.024716,0.023720,0.049912,0.098739,0.157074,0.092067,0.010097,0
1,u108690,0.000000,0.017720,0.141401,0.028406,0.164466,0.002050,0.027290,0.079994,0.078703,...,0.033342,0.008906,0.057805,0.006797,0.029466,0.013553,0.031181,0.077544,0.005561,1
2,u108339,0.006854,0.136171,0.072904,0.044623,0.251824,0.004261,0.023074,0.063888,0.079558,...,0.015415,0.025124,0.051542,0.026707,0.007515,0.029399,0.041229,0.056211,0.000000,1


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [ ]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [ ]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.11656586, 0.05995532, 0.31659145, 0.20594313, 0.03621961,
       0.03485763, 0.04833336, 0.05972265, 0.05739987, 0.15679439])

In [ ]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [ ]:
from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             classification_report, precision_recall_curve, confusion_matrix)

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.2649104222992187, F-Score=0.743, Precision=0.696, Recall=0.796


In [ ]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9633094947380663

In [ ]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.26491,0.742857,0.696429,0.795918,0.963309


2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог)

In [ ]:
FUNC = np.median

In [ ]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.0,0.000000,0.032064,0.000000,0.099142,0.0,0.000000,0.084580,0.034143,...,0.021183,0.028889,0.000000,0.000000,0.000000,0.0,0.079513,0.052455,0.000000,0.0
1,u108690,0.0,0.000000,0.144295,0.005132,0.122968,0.0,0.000000,0.053632,0.066017,...,0.146249,0.024664,0.000000,0.067338,0.000000,0.0,0.006699,0.027790,0.044706,0.0
2,u108339,0.0,0.013738,0.075189,0.000000,0.211210,0.0,0.007073,0.061886,0.091712,...,0.044745,0.017949,0.006803,0.028353,0.021218,0.0,0.013871,0.005482,0.066490,0.0


In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.0,0.000000,0.032064,0.000000,0.099142,0.0,0.000000,0.084580,0.034143,...,0.028889,0.000000,0.000000,0.000000,0.0,0.079513,0.052455,0.000000,0.0,0
1,u108690,0.0,0.000000,0.144295,0.005132,0.122968,0.0,0.000000,0.053632,0.066017,...,0.024664,0.000000,0.067338,0.000000,0.0,0.006699,0.027790,0.044706,0.0,1
2,u108339,0.0,0.013738,0.075189,0.000000,0.211210,0.0,0.007073,0.061886,0.091712,...,0.017949,0.006803,0.028353,0.021218,0.0,0.013871,0.005482,0.066490,0.0,1


In [ ]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [ ]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.06530658, 0.08502676, 0.50058763, 0.2690722 , 0.06493959,
       0.02876773, 0.03844834, 0.03916434, 0.05187884, 0.14661138])

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.2751430837362999, F-Score=0.802, Precision=0.778, Recall=0.829


In [ ]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9749404035118321

In [ ]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.264910,0.742857,0.696429,0.795918,0.963309
1,median,0.275143,0.802372,0.777778,0.828571,0.974940



3. Повторить п.2, но используя уже не медиану, а max


In [ ]:
FUNC = np.max

In [ ]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.052510,0.058850,0.083398,0.194922,0.363387,0.201383,0.118000,0.169094,0.155447,...,0.091094,0.063629,0.025840,0.090374,0.092058,0.169125,0.192396,0.527625,0.330287,0.036889
1,u108690,0.000000,0.066142,0.245367,0.140920,0.509875,0.012301,0.145613,0.295562,0.194415,...,0.450245,0.057095,0.027375,0.101806,0.040781,0.176795,0.046979,0.065388,0.259398,0.018991
2,u108339,0.023817,0.420559,0.150047,0.144908,0.472098,0.025566,0.074897,0.126932,0.133996,...,0.098275,0.026078,0.110914,0.188660,0.093496,0.030449,0.116623,0.166770,0.115964,0.000000


In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.052510,0.058850,0.083398,0.194922,0.363387,0.201383,0.118000,0.169094,0.155447,...,0.063629,0.025840,0.090374,0.092058,0.169125,0.192396,0.527625,0.330287,0.036889,0
1,u108690,0.000000,0.066142,0.245367,0.140920,0.509875,0.012301,0.145613,0.295562,0.194415,...,0.057095,0.027375,0.101806,0.040781,0.176795,0.046979,0.065388,0.259398,0.018991,1
2,u108339,0.023817,0.420559,0.150047,0.144908,0.472098,0.025566,0.074897,0.126932,0.133996,...,0.026078,0.110914,0.188660,0.093496,0.030449,0.116623,0.166770,0.115964,0.000000,1


In [ ]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [ ]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.0526981 , 0.00465111, 0.29595987, 0.33951549, 0.01147503,
       0.02328607, 0.03405388, 0.02657765, 0.03783612, 0.14670363])

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.2976508080819808, F-Score=0.700, Precision=0.658, Recall=0.747


In [ ]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9429594743880458

In [ ]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.264910,0.742857,0.696429,0.795918,0.963309
1,median,0.275143,0.802372,0.777778,0.828571,0.974940
2,amax,0.297651,0.699809,0.658273,0.746939,0.942959


4. *Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (взяв список новостей пользователя)
	- подсказка 1: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал
	- подсказка 2: нужен именно idf, как вес.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
users['articles_str'] = users['articles'].apply(lambda x: x.replace('[','').replace(']', '').replace(',', ''))

users['articles_str'].iloc[0]

'293672 293328 293001 293622 293126 1852'

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(users['articles_str'])

TfidfVectorizer()

In [ ]:
idf = pd.DataFrame({'article_id': tfidf.get_feature_names_out(),
                    'idf': tfidf.idf_})

idf

,article_id,idf
0,10,8.888710
1,100,7.907880
2,1000,8.041412
3,1001,8.888710
4,1002,8.888710
...,...,...
14776,995,8.377884
14777,996,8.195562
14778,997,8.601027
14779,998,9.294175


In [ ]:
def get_user_embedding_idf(user_articles_list, doc_dict):
    user_articles_list = eval(user_articles_list)
    
    user_vector = np.zeros((len(user_articles_list), N_topic))
    for i, doc_id in enumerate(user_articles_list):
        try:
            weight = idf[idf['article_id'] == str(doc_id)]['idf'].values[0]
        except Exception as e:
            weight = 0
        user_vector[i] = doc_dict[doc_id] * weight

    user_vector = np.median(user_vector, axis=0)
    return user_vector

In [ ]:
from tqdm import tqdm
tqdm.pandas()

user_embeddings = pd.DataFrame([i for i in users['articles'].progress_apply(lambda x: get_user_embedding_idf(x, doc_dict))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

100%|██████████| 8000/8000 [01:18<00:00, 102.19it/s]


,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.0,0.000000,0.275787,0.000000,0.892010,0.0,0.000000,0.704685,0.317335,...,0.182197,0.248473,0.000000,0.000000,0.000000,0.0,0.711480,0.445282,0.000000,0.0
1,u108690,0.0,0.000000,1.219619,0.044138,1.128004,0.0,0.000000,0.487864,0.575293,...,1.271430,0.208643,0.000000,0.591662,0.000000,0.0,0.057614,0.241923,0.389916,0.0
2,u108339,0.0,0.118977,0.634690,0.000000,1.707213,0.0,0.065738,0.522747,0.811503,...,0.377093,0.150829,0.063225,0.234991,0.185512,0.0,0.116206,0.047150,0.550475,0.0


In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.0,0.000000,0.275787,0.000000,0.892010,0.0,0.000000,0.704685,0.317335,...,0.248473,0.000000,0.000000,0.000000,0.0,0.711480,0.445282,0.000000,0.0,0
1,u108690,0.0,0.000000,1.219619,0.044138,1.128004,0.0,0.000000,0.487864,0.575293,...,0.208643,0.000000,0.591662,0.000000,0.0,0.057614,0.241923,0.389916,0.0,1
2,u108339,0.0,0.118977,0.634690,0.000000,1.707213,0.0,0.065738,0.522747,0.811503,...,0.150829,0.063225,0.234991,0.185512,0.0,0.116206,0.047150,0.550475,0.0,1


In [ ]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [ ]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([9.60378497e-03, 3.97222935e-03, 9.44610248e-01, 1.38776363e-01,
       1.48165273e-03, 4.59826987e-04, 1.94774305e-03, 1.17602060e-04,
       6.40865805e-04, 3.64668223e-02])

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.4933609655858414, F-Score=0.858, Precision=0.894, Recall=0.824


In [ ]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9885993371707658

In [ ]:
metrics_df = metrics_df.append({
    'model': 'idf_median',
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)


5. Сформировать на выходе единую таблицу, сравнивающую качество 2/3 разных метода получения эмбедингов пользователей: median, max, idf_mean по метрикам roc_auc, precision, recall, f_score


In [ ]:
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.264910,0.742857,0.696429,0.795918,0.963309
1,median,0.275143,0.802372,0.777778,0.828571,0.974940
2,amax,0.297651,0.699809,0.658273,0.746939,0.942959
3,idf_mean,0.493361,0.857749,0.893805,0.824490,0.988599


6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных